# Sparkify ETL Pipeline

A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. Currently, there is no easy way to query the data to generate the results, since the data reside in a directory of CSV files on user activity on the app.

## First: ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we will work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing Apache Cassandra code

#### Creating an Apache Cassandra Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace for the tables

In [8]:
try:
    # Creating keyspace with the name "sparkify" and setting it's replication strategy
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem

#### Setting  the Keyspace to sparkify keyspace

In [9]:
try:
    # setting the session to the keyspace "sparkify"
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem

### We need to create tables to run the queries below

### Task 1: Getting the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 1. Creating the table for this query

#### In this query we considered <font color="red">sessionId</font> and <font color="red">itemInSession</font> as a composite key because <font color="blue">it appears in the where clause to be searched and make the primary key unique</font>

In [11]:
try:
    session.execute("Create table if not exists songs_heard(\
                    sessionId int,\
                    itemInSession int,\
                    artist text,\
                    song_title text,\
                    length decimal,\
                    primary key(sessionId,itemInSession)\
                    )")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem                

#### 2. Copying the required data from <font color="red">event_datafile_new.csv</font> and inserting it into to the table created

In [12]:
file = 'event_datafile_new.csv'

# Opening the file to get the data from it
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Assigning the INSERT statements into the `query` variable
        query = "Insert into songs_heard (sessionId,ItemInSession,artist,song_title,length)"
        query = query + "values (%s,%s,%s,%s,%s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.

        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

#### 3. Doing a SELECT to verify that the data have been inserted into the table

#### The query gets the artist name, the song title and its length from the table in the session number 338 and the item number 4 in that session

In [13]:
# Task 1 : Getting the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession  = 4
try:
    rows = session.execute("select artist,song_title,length from songs_heard where sessionId = 338 and itemInSession = 4")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem    

# showing the data from the query as values    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Task 2: Getting only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### 1. Creating the table for this query

#### In this query we considered <font color="red">userId</font> and <font color="red">sessionId</font> as a composite key because <font color="blue">it appears in the where clause to be searched and it uniquly identify every single row</font> and we used <font color="red">itemsInSession</font> as clustring column to order the songs

In [14]:
# Creating a table with UserId then sessionId and itemInSession as primary key because we searched with them in the where clause
# and it has to be with the same order as the query
# also used itemInSession as clustring column to order the songs
try:
    session.execute("create table if not exists user_song(\
                     userId int,\
                     sessionId int,\
                     itemInSession text,\
                     artist text,\
                     firstName text,\
                     lastName text,\
                     song_title text,\
                     primary key((userId,sessionId),itemInSession))\
                     WITH CLUSTERING ORDER BY (itemInSession ASC)")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem                  

#### 2. Copying the required data from <font color="red">event_datafile_new.csv</font> and inserting it into to the table created

In [15]:
file = 'event_datafile_new.csv'

# Opening the file to get the data from it
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigning the INSERT statements into the `query` variable
        query = "Insert into user_song (UserId,SessionId,itemInSession,artist,firstName,lastName,song_title)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]),line[3],line[0],line[1],line[4],line[9]))

#### 3. Doing a SELECT to verify that the data have been inserted into the table

#### The query gets the artist name, the song title ,the first name and the last name from the table depending on the user ID and the sessionId
#### Also the result will be sorted by the itemInSession

In [16]:
## Task 2: Getting the name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

try:
    rows = session.execute("select artist,song_title,firstName,lastName from user_song where userId = 10 and sessionId = 182")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem   

    
# showing the data from the query as values  
for row in rows:
    print (row.artist, row.song_title, row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Task 3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### 1. Creating the table for this query

#### In this query we considered <font color="red">song_title</font> and <font color="red">user_id</font> as a composite key because <font color="blue">it uniquly identify every single row and the song_title is querable in the where clause</font>

In [21]:
try:
    session.execute("create table if not exists song_listener(\
    song_title text,\
    userId int,\
    firstName text,\
    lastName text,\
    primary key(song_title,userId))")
except Exception as e:
    print(e)
    # If we have a problem we will get an execption which describe the problem  

#### 2. Copying the required data from <font color="red">event_datafile_new.csv</font> and inserting it into to the table created

In [23]:
file = 'event_datafile_new.csv'

# Opening the file to get the data from it
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
# Assigning the INSERT statements into the `query` variable
        query = "Insert into song_listener (song_title,userId,firstName,lastName)"
        query = query + "values (%s,%s,%s,%s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### 3. Doing a SELECT to verify that the data have been inserted into the table

#### The query retreives the first and last name for every user listened to 'All Hands Against His Own' song

In [25]:
## Task 3: geting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    rows = session.execute("select firstName,lastName from song_listener where song_title = 'All Hands Against His Own' ")
except Exception as e:
    print(e)
for row in rows:
    print (row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [26]:
## Dropping the table before closing out the sessions
try:
    session.execute("drop table if exists artist_song")
except Exception as e:
    print(e)

In [27]:
try:
    session.execute("drop table if exists user_song")
except Exception as e:
    print(e)

In [28]:
try:
    session.execute("drop table if exists song_listener")
except Exception as e:
    print(e)

### Closing the session and cluster connection

In [29]:
session.shutdown()
cluster.shutdown()

<p align="center">Congrats🥳</p>